In [1]:
import numpy as np
import wisps
import wisps.simulations as wispsim
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm 
import pandas as pd
import matplotlib as mpl
import wisps.simulations.effective_numbers as eff
import astropy.coordinates as astrocoord
from wisps import drop_nan
import seaborn as sns
%matplotlib inline
import splat

Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database
Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database
Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database


###Step by step checking

1. Distances before and after cuts 

In [ ]:
d150=pd.read_pickle(wisps.OUTPUT_FILES+'/distance_samples{}'.format(150))
d500=pd.read_pickle(wisps.OUTPUT_FILES+'/distance_samples{}'.format(500))

In [ ]:
wispsim

In [ ]:
cands=pd.read_pickle(wisps.LIBRARIES+'/real_ucds.pkl')


In [ ]:
#(d500[17][-1]).min()

In [ ]:
fig, ax=plt.subplots()
h=plt.hist(d150[17][-1], bins=32, histtype='step', density=True)
h=plt.hist(d500[17][-1], bins=32, histtype='step', density=True)
#plt.hist(cands.distance.apply(np.log10), density=True,  histtype='step' )
#sns.kdeplot(cands.distance.apply(np.log10), ax=ax, label='observed', c='k')


In [ ]:
cmap_teff=sns.diverging_palette(100, 300, s=80, l=55, n=19, as_cmap=True)

In [ ]:
#older_pnts=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_corrected.pkl')
#older_pnts=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_unccorrected.pkl')

pnts=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_correctedf110.pkl')
#pnts140=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_correctedf140.pkl')
#pnts160=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_correctedf160.pkl')

In [ ]:
def bin_volumes(pnts):
    volumes=[]
    dlimits=[]
    for pnt in pnts:
        vs=[]
        dls=[]
        for g in wispsim.SPGRID:
            vsx=[]
            for h in wispsim.HS:
                vsx.append((pnt.volumes[h])[g])
            dls.append(pnt.dist_limits[g])
            vs.append(vsx)
        volumes.append(vs)
        dlimits.append(dls)
    volumes=np.array(volumes)
    dlimits=np.array(dlimits)
    
    return  volumes, dlimits

In [ ]:
volumes110, _ =bin_volumes(pnts)
#oldervolumes, _=bin_volumes(older_pnts)
#volumes140, _=bin_volumes(pnts140)
#volumes160, _=bin_volumes(pnts160)

In [ ]:
#fig = plt.figure(figsize=(8,5))
#ax = fig.add_subplot(111, projection="mollweide")
#c=ax.scatter(galc.l.wrap_at(180*u.degree).radian,galc.b.wrap_at(90*u.degree).radian, marker='+', cmap='viridis')
#ax.set_xlabel('l (deg)', fontsize=18)
#ax.set_ylabel('b (deg)', fontsize=18)
#plt.grid()
#plt.savefig(wisps.OUTPUT_FIGURES+'/fields_skymap.pdf', bbox_inches='tight')

In [ ]:
from matplotlib.colors import Normalize


In [ ]:
from scipy import integrate

In [ ]:
volume_fx=np.vectorize(wispsim.custom_volume)

In [ ]:
import seaborn as sns
import matplotlib
#cmap= sns.color_palette("coolwarm", 8, as_cmap=True)
cmap=matplotlib.cm.get_cmap('coolwarm')
cnorm=Normalize(wispsim.HS[0]/100, (wispsim.HS[-1])/100)

In [ ]:
ds=np.logspace(0, 3.5, 1000)
fig, ax=plt.subplots(figsize=(8, 5))
for idx, h in tqdm(enumerate(wispsim.HS)):
    plt.plot(ds, np.log10(volume_fx(0.,np.pi/4, 0, ds,h)), color=cmap(cnorm(h/100)), label=r'h ={}'.format(h))
plt.plot(ds, np.log10(ds**3), c='k', label=r'd$^3$')
plt.ylabel(r'Log Veff (pc$^3$)', fontsize=18)
plt.xlabel('d (pc)', fontsize=18)
plt.legend(fontsize=14)

In [ ]:
VOLUMESF110=np.nansum(volumes110.T, axis=2)*4.1*(u.arcmin**2).to(u.radian**2)
#OLDERVOLUMES=np.nansum(oldervolumes.T, axis=2)*4.1*(u.arcmin**2).to(u.radian**2)
#VOLUMESF140=np.nansum(volumes140.T, axis=2)*4.1*(u.arcmin**2).to(u.radian**2)
#VOLUMESF160=np.nansum(volumes160.T, axis=2)*4.1*(u.arcmin**2).to(u.radian**2)

In [ ]:
h=500

In [ ]:
cands=pd.read_pickle(wisps.LIBRARIES+'/real_ucds.pkl')

In [ ]:
mag_f=pd.DataFrame.from_records([x.mag_limits for x in pnts])

In [ ]:
model='baraffe2003'
hidx=400
cutdf=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str(model)+str('h')+str(hidx)+'F110_corrected')

In [ ]:
spts=cutdf.spt.values
scales=(wisps.POLYNOMIAL_RELATIONS['mag_limit_corrections']['F110W'])[0](spts)

In [ ]:
fig, ax=plt.subplots(figsize=(8, 5))
for idx, h in enumerate(wispsim.HS):
    #plt.plot(wispsim.SPGRID, np.log10(OLDERVOLUMES[idx]), color=cmap(cnorm(h/100)),
     #   linewidth=3, linestyle='--', alpha=0.3)
    
    plt.plot(wispsim.SPGRID, np.log10(VOLUMESF110[idx]), color=cmap(cnorm(h/100)),
             linewidth=3, label=r'h ={} pc'.format(h), linestyle='-')

#plt.plot(wispsim.SPGRID, np.log10(vsunif), label=r'd$^3$' )
plt.ylabel(r'Log  Veff', fontsize=18)
plt.xlabel('SpT', fontsize=18)
#plt.legend(fontsize=14)
cax = fig.add_axes([.3, 0.4, .3, 0.03])
mp=matplotlib.cm.ScalarMappable(norm=cnorm, cmap=cmap)
cbar=plt.colorbar(mp, cax=cax, orientation='horizontal')
cbar.ax.set_xlabel(r' H (pc)', fontsize=18)
cax.minorticks_on()
plt.minorticks_on()
#plt.grid(which='both')

plt.tight_layout()
ax.set_xticks([20, 25, 30, 35, 40])
ax.set_xticklabels(['L0', 'L5', 'T0', 'T5', 'Y0'])

plt.savefig(wisps.OUTPUT_FIGURES+'/simulation_volumes.pdf', bbox_inches='tight')

In [ ]:
np.nansum(VOLUMESF110[0][:3])*(10**-3)

In [ ]:
model='baraffe2003'
hidx=500
fig, ax=plt.subplots()
for hidx in wispsim.HS:
    cutdf=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str(model)+str('h')+str(hidx)+'F110_corrected')
    #cutdf=cutdf[cutdf.snr >= 3.]
    sns.kdeplot(cutdf.d.apply(np.log10), label='simulated h= {} pc'.format(hidx), ax=ax)
    
cands=cands[cands.snr1 >=3.]
sns.kdeplot(cands.distance.apply(np.log10), label='observed', c='k')

plt.hist(wisps.drop_nan(cands.distance.apply(np.log10)), density=True)
#plt.legend()
ax.set_xlim([2, 4])
plt.xlabel('Log Distance')
plt.xscale('log')
plt.ylabel('Normalized Number')

In [ ]:
model='baraffe2003'
fig, ax=plt.subplots(ncols=2, figsize=(12, 6))
for idx, hidx in enumerate([350, 1000]):
    cutdf=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str(model)+str('h')+str(hidx)+'F110_corrected')
    #cutdf=cutdf[cutdf.snr >= 3.]
    ax[idx].scatter(cutdf.spt, cutdf.d, c=cutdf.sl, s=5, alpha=.5, cmap='viridis')
    ax[idx].errorbar(cands.spt,cands.distance,   label='observed', c='k',  fmt='o', alpha=1.)
    ax[idx].set_yscale('log')
    ax[idx].set_ylabel('Distance (pc)')
    ax[idx].set_xlabel('SpT')
    ax[idx].set_title('h = {} pc'.format(hidx), fontsize=18)

In [ ]:
cutdf=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str('baraffe2003')+str('h')+str(1000)+'F110_corrected')


In [ ]:
h=plt.hist(cutdf.appf110, density=True, bins=32)
plt.hist(cands.F110W, density=True,  histtype='step' )

In [ ]:
h=plt.scatter(cutdf.appf110, np.log10(cutdf.f110_unc), s=10)# density=True, bins=32)

In [ ]:
x=[2, 3]
print ('{}{}'.format(*x))

In [ ]:
h=plt.scatter(cutdf.appf110, np.log10(cutdf.snrj110), s=10)
# density=True, bins=32)

In [ ]:
c=plt.scatter(cutdf.snr.apply(np.log10), cutdf.appf140, s=5, 
              alpha=0.5, marker='+', c=cutdf.spt, cmap='coolwarm')
#is this binning a problem?
plt.xlabel('LOG J-SNR')
plt.ylabel('SpT')
plt.axvline(np.log10(3.))
plt.colorbar(c)
plt.minorticks_on()

In [ ]:
cutdf['spt_r']=cutdf.spt.apply(np.round)

In [ ]:
plt.plot([x[1].spt_r.mean() for x in cutdf.groupby('spt_r')], 
         [x[1].sl.mean() for x in cutdf.groupby('spt_r')])

In [ ]:
wcands=cands[cands.grism_id.str.lower().str.startswith('par')]
hcands=cands[~cands.grism_id.str.lower().str.startswith('par')]

In [ ]:
#sns.kdeplot(cutdf.appF140, cutdf.snr.apply(np.log10), s=50, marker=',', alpha=0.01)
#sns.kdeplot(cutdf.appF140, cutdf.snr.apply(np.log10),  fill=True)
sns.scatterplot(cutdf.appf140, cutdf.spt, s=5, 
                color='#AAAAAA', alpha=0.7)
#sns.histplot(x=cutdf.appF140, y=cutdf.snr.apply(np.log10), bins=50, pthresh=.1, cmap="mako")
sns.kdeplot(cutdf.appf140, cutdf.spt, levels=np.flip(1-np.array([ 0.5, 0.75, 0.89, 0.94, 0.96])), 
            color="#001f3f", linewidths=1)

plt.errorbar(wcands.F140W, wcands.spt, xerr=wcands.F140W_er, yerr=wcands.spt_er, fmt='o', c='k', label='WISP')
plt.errorbar(hcands.F140W, hcands.spt, xerr=hcands.F140W_er, yerr=hcands.spt_er, fmt='o', c='#B10DC9', 
             label='HST-3D')


#is this binning a problem?
plt.xlabel('F140W')
plt.legend()
plt.ylabel('SpT')
plt.minorticks_on()
#plt.colorbar(c)

#plt.minorticks_on()

In [ ]:
#pd.DataFrame.from_records(cutdf.pnt.apply(lambda x: x.mag_limits).values)

In [ ]:
plt.scatter(cutdf.pnt.apply(lambda x: np.log10(x.imag_exptime/1000)), cutdf.f140,
            alpha=0.1)

In [ ]:
c=plt.scatter(cutdf.appf140, cutdf.spt, s=10,marker='.', alpha=0.1)

In [ ]:
volumes=[]
dlimits=[]
for pnt in pnts:
    vs=[]
    dls=[]
    for g in wispsim.SPGRID:
        vsx=[]
        for h in wispsim.HS:
            vsx.append((pnt.volumes[h])[g])
        dls.append(pnt.dist_limits[g])
        vs.append(vsx)
    volumes.append(vs)
    dlimits.append(dls)
volumes=np.array(volumes)
dlimits=np.array(dlimits)

steps=np.arange(len(pnts))
AREA=4.1*(u.arcmin**2).to(u.radian**2)

In [ ]:
fig, ((ax, ax1), (ax2, ax3))=plt.subplots(figsize=(10, 8), ncols=2, nrows=2)
for idx, h in enumerate(wispsim.HS):
    ax.step(steps, np.log10(np.cumsum(volumes[:, 0, idx])*AREA), color=cmap(cnorm(h/100)), linewidth=3)
    ax1.step(steps, np.log10(np.cumsum(volumes[:, 9, idx])*AREA), color=cmap(cnorm(h/100)), linewidth=3)
    ax2.step(steps, np.log10(np.cumsum(volumes[:,-6 , idx])*AREA), color=cmap(cnorm(h/100)), linewidth=3)
    ax3.step(steps, np.log10(np.cumsum(volumes[:, -1, idx])*AREA), color=cmap(cnorm(h/100)), linewidth=3, label=r'h ={}'.format(h))


ax.set_ylabel(r'Log Cumulative Veff (pc$^3$)', fontsize=18)
ax2.set_ylabel(r'Log Cumulative Veff (pc$^3$)', fontsize=18)

for a in [ax, ax1, ax2, ax3]:
    a.set_xlabel('Number of Pointings', fontsize=18)
    a.minorticks_on()
    
ax.set_title('{} UCDs'.format(splat.typeToNum(wispsim.SPGRID[0])), fontsize=18)
ax1.set_title('{} UCDs'.format(splat.typeToNum(wispsim.SPGRID[9])), fontsize=18)
ax2.set_title('{} UCDs'.format(splat.typeToNum(wispsim.SPGRID[-6])), fontsize=18)
ax3.set_title('{} UCDs'.format(splat.typeToNum(wispsim.SPGRID[-1])), fontsize=18)

#ax3.legend(fontsize=15)
cax = fig.add_axes([.6, 0.2, .3, 0.03])
mp=matplotlib.cm.ScalarMappable(norm=cnorm, cmap=cmap)
cbar=plt.colorbar(mp, cax=cax, orientation='horizontal')
cbar.ax.set_xlabel(r' H (pc)', fontsize=18)
cax.minorticks_on()

plt.tight_layout()


plt.savefig(wisps.OUTPUT_FIGURES+'/cumulative_volumes.pdf', bbox_inches='tight')

In [ ]:
import numba

In [ ]:
@numba.jit
def get_kde(r, z):
    kde=wisps.kernel_density([r, z])
    dens=kde.pdf([r, z])
    return dens

In [ ]:
cutdf150=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str(model)+str('h')+str(150)+'F110_corrected')
cutdf150['ra']=cutdf150.pnt.apply(lambda x: x.coord.ra)
cutdf150['dec']=cutdf150.pnt.apply(lambda x: x.coord.dec)
galc150=SkyCoord(ra=cutdf150['ra'].values, dec=cutdf150['dec'].values, distance= cutdf150.d.values*u.pc)

cutdf400=pd.read_hdf(wisps.OUTPUT_FILES+'/final_simulated_sample_cut.h5', key=str(model)+str('h')+str(350)+'F110_corrected')
cutdf400['ra']=cutdf400.pnt.apply(lambda x: x.coord.ra)
cutdf400['dec']=cutdf400.pnt.apply(lambda x: x.coord.dec)
galc400=SkyCoord(ra=cutdf400['ra'].values, dec=cutdf400['dec'].values, distance= cutdf400.d.values*u.pc)

In [ ]:
mask1=(cutdf400.spt>=20).values
#mask1=

In [ ]:
#mask0

In [ ]:
galoc0=galc150.transform_to(astrocoord.Galactocentric(galcen_distance=8.3*u.kpc))
x0, y0, z0=galoc0.cartesian.xyz

galoc1=galc400.transform_to(astrocoord.Galactocentric(galcen_distance=8.3*u.kpc))
x1, y1, z1=galoc1.cartesian.xyz

d0=(x0**2+y0**2+z0**2)**(0.5)
r0=(x0**2+y0**2)**(0.5)

d1=(x1**2+y1**2+z1**2)**(0.5)
r1=(x1**2+y1**2)**(0.5)


realcoords=SkyCoord(cands.coord.values)

galoc=realcoords.transform_to(astrocoord.Galactocentric(galcen_distance=8.3*u.kpc))
x, y, z=galoc.cartesian.xyz

r=(x**2+y**2)**0.5

In [ ]:
#mask0= 

In [ ]:
fig, ax=plt.subplots( figsize=(10, 6), ncols=2, sharey='row')


h=ax[0].hist(r1.flatten().value[~mask1], 
          bins=32, histtype='step', density=True, log=True, linewidth=3)
h=ax[0].hist(r1.flatten().value[mask1],
          bins=32, histtype='step',  density=True, log=True, linewidth=3)
#h=ax[0].hist(d1.flatten().value,  color='#0074D9', bins=32, density=True,
#          histtype='step', label="h = 400 pc", log=True)

h=ax[1].hist(z1.flatten().value[~mask1], bins=32, density=True,
         histtype='step', log=True, label='M', linewidth=3)

h=ax[1].hist(z1.flatten().value[mask1],  bins=32, density=True,
         histtype='step',  log=True, label='L \&T', linewidth=3)
ax[1].legend()

#h=ax[1].hist(z1.flatten().value,  color='#0074D9', bins=32, density=True,  
#          histtype='step', label="h = 400 pc", log=True)

#fig, ax=plt.subplots( figsize=(10, 10))
#h=ax.hist(z0.flatten().value, color='#FF4136',
#          bins='auto', histtype='step', label="h = 200 pc")
#
#h=ax.hist(z1.flatten().value,  color='#0074D9', bins='auto', 
#          histtype='step', label="h = 400 pc")
#c1=ax1[0].scatter(np.log10(r0), z0, c=cutdf150.spt.values, vmin=17, vmax= 40, cmap='coolwarm', 
#                  s=1, alpha=0.5)

#ax1[0].set_title("h = 200 pc", fontsize=18)
#ax1[1].set_title("h = 400 pc", fontsize=18)

#c=ax1[1].scatter(np.log10(r1), z1, c=cutdf400.spt.values,  vmin=17, vmax= 40, 
#                 cmap='coolwarm',   s=1, alpha=0.5)

#cbar0=plt.colorbar(c, ax=ax1[1],    orientation='horizontal')
#cbar1=plt.colorbar(c1, ax=ax1[0],    orientation='horizontal')

#sc=ax1[1].scatter(np.log10(r), z, c=cands.spt.values, s=50, cmap='coolwarm',  marker='*', edgecolors='#111111', 
#                   vmin=17, vmax= 40, label='Real UCDs')
#sc=ax1[0].scatter(np.log10(r), z, c=cands.spt.values, s=50, cmap='coolwarm',  marker='*', edgecolors='#111111', 
#                  vmin=17, vmax= 40,label='Real UCDs')

#cbar0.ax.set_xlabel(r'Spectral Type', fontsize=18)
#cbar1.ax.set_xlabel(r'Spectral Type', fontsize=18)
#cbar1.ax.set_xlabel(r'Log $ \rho/ \rho_0$ ', fontsize=18)

#ax[0].legend(fontsize=15, loc='lower left')
#ax[0].set_xlabel('Log Distance (pc)', fontsize=18)
#ax[0].set_ylabel('N', fontsize=18)

#ax[1].set_xlabel('Z (pc)', fontsize=18)
#ax[1].set_ylabel('N', fontsize=18)
#ax[1].set_yscale('log')

#ax1[0].legend(fontsize=15, loc='lower left')
#ax1[0].set_xlabel('R (pc)', fontsize=18)
#ax1[0].set_ylabel('Z (pc)', fontsize=18)

#ax1[1].legend(fontsize=15, loc='lower left')
#ax1[1].set_xlabel('R (pc)', fontsize=18)
#ax1[1].set_ylabel('Z (pc)', fontsize=18)

#ax1[0].set_xticks(np.array([10, 100, 500, 1000]))#, major=True)
#ax1[1].set_xticks([10, 100, 500, 1000], major=True)

#ax1[0].set_xscale('log')
#ax1[1].set_xscale('log')

ax[0].axvline(8300, c='k', linestyle='--')
ax[-1].axvline(27, c='k', linestyle='--')

for a in ax:
    a.minorticks_on()
    #a.set_xscale('log')
    
#cbar0.ax.set_xticks([17, 20, 25, 30, 35])
#cbar0.ax.set_xticklabels(['M7', 'L0', 'L5', 'T0', 'T5'])

#cbar1.ax.set_xticks([17, 20, 25, 30, 35])
#cbar1.ax.set_xticklabels(['M7', 'L0', 'L5', 'T0', 'T5'])
#plt.tight_layout()
plt.savefig(wisps.OUTPUT_FIGURES+'/simulation_distances.jpeg', bbox_inches='tight', rasterized=True)

In [ ]:
gh

In [ ]:
corr_pols=wisps.POLYNOMIAL_RELATIONS['mag_limit_corrections'] 

In [ ]:
wispsim.MAG_LIMITS['ncutoff']

In [ ]:
def less_than_fifty(row):
    ser={'lF110':False, 'lF140':False, 'lF160':False}
    
    if row.nF110 <wispsim.MAG_LIMITS['ncutoff']:
        ser['lF110']=True
    if row.nF140 <wispsim.MAG_LIMITS['ncutoff']:
        ser['lF140']=True
    if row.nF160 <wispsim.MAG_LIMITS['ncutoff']:
        ser['lF160']=True
    return pd.Series(ser)

maglimits=pd.DataFrame([x.mag_limits for x in pnts])
maglimits['imexp']=[x.imag_exptime for x in pnts]
#exposure_times=[np.nanmean(x.exposure_time) for x in pnts]
maglimits['pointing']=[x.name for x in pnts]
maglimits['survey']=[x.survey for x in pnts]
#maglimits['exp']=np.log10(exposure_times)

nsrcs=pd.DataFrame([x.number_of_sources  for x in pnts])
number_of_sources=nsrcs.rename(columns={'F140': 'nF140', 'F160': 'nF160', 'F110':'nF110'})
less_than_df=number_of_sources.apply(less_than_fifty, axis=1)
number_of_sources['pointing']=[x.name for x in pnts]
number_of_sources=number_of_sources.join(less_than_df)
mag_lts_df=maglimits.merge(number_of_sources, on='pointing')


In [ ]:
mag_lts_df.imexp.median(), mag_lts_df.imexp.std()

In [ ]:
wispmglts=mag_lts_df[mag_lts_df.survey=='wisps']
hst3dpmglts=mag_lts_df[mag_lts_df.survey!='wisps']

In [ ]:
hst3dpmglts.F140.max()

In [ ]:
wispmglts.F110.max()

In [ ]:
(wispsim.MAG_LIMITS['hst3d']['F140'][0])(np.log(812))

In [ ]:
fig, ax=plt.subplots(ncols=3, figsize=(12, 4),  sharex=True, sharey=True)

ax[0].scatter(wispmglts['F110'][wispmglts.lF110.apply(bool)],
              np.log10(wispmglts.imexp)[wispmglts.lF110.apply(bool)], 
         facecolors='none', edgecolors='r', marker='o')
ax[0].scatter(wispmglts['F110'][~wispmglts.lF110.apply(bool)],
              np.log10(wispmglts.imexp)[~wispmglts.lF110.apply(bool)], 
         marker='+', color='#111111')
ax[0].plot( wispsim.MAG_LIMITS['wisps']['F110'][0](np.arange(2, 4, 0.1)),
              np.arange(2, 4, 0.1), color='#111111')

ax[1].scatter(wispmglts['F140'][wispmglts.lF140.apply(bool)],
              np.log10(wispmglts.imexp)[wispmglts.lF140.apply(bool)], 
           facecolors='none', edgecolors='r', marker='o')
ax[1].scatter(wispmglts['F140'][~wispmglts.lF140.apply(bool)],
              np.log10(wispmglts.imexp)[~wispmglts.lF140.apply(bool)], 
           marker='+', color='#111111')
ax[1].plot( wispsim.MAG_LIMITS['wisps']['F140'][0](np.arange(2, 4, 0.1)),
              np.arange(2, 4, 0.1),  color='#111111')
ax[1].scatter(hst3dpmglts['F140'], np.log10(hst3dpmglts.imexp),  
              marker='^', color='#111111')

#ax[1][1].scatter(hst3dpmglts['F140'], np.log10(hst3dpmglts.imexp), 
#           c=hst3dpmglts.lF140.apply(int), cmap='coolwarm', marker='o')

ax[2].scatter(wispmglts['F160'][wispmglts.lF160.apply(bool)],
              np.log10(wispmglts.imexp)[wispmglts.lF160.apply(bool)], 
             facecolors='none', edgecolors='r', marker='o',
             label='WISP fits')

ax[2].scatter(wispmglts['F160'][~wispmglts.lF160.apply(bool)],
              np.log10(wispmglts.imexp)[~wispmglts.lF160.apply(bool)], 
             marker='+',  color='#111111', label='WISP')
ax[2].plot( wispsim.MAG_LIMITS['wisps']['F160'][0](np.arange(2, 4, 0.1)),
              np.arange(2, 4, 0.1),  color='#111111')

ax[2].scatter(hst3dpmglts['F160'], np.log10(hst3dpmglts.imexp),  
              marker='^', label='3D-HST', color='#111111')
ax[2].legend(fontsize=12)

#ax[1][1].set_ylim([2.8, 2.95])
#ax[1][2].set_ylim([2.8, 2.95])
for a in ax:
    a.minorticks_on()
    a.set_ylabel('Log Exposure Time (s)')

ax[0].set_xlabel(r'F110W $_{\mathrm{lim}}$')
ax[1].set_xlabel(r'F140W $_{\mathrm{lim}}$')
ax[2].set_xlabel(r'F160W $_{\mathrm{lim}}$')

plt.savefig(wisps.OUTPUT_FIGURES+'/mag_limit.pdf', 
            bbox_inches='tight', rasterized=True)

In [ ]:
from scipy import stats
def get_random_pointing(survey, key):
    ps=[x for x in pnts if x.survey.startswith(survey)]
    p=pnts[np.random.randint(len(ps))]
    vals=wisps.drop_nan(np.vstack(p.mags[key])[:,0])
    print (p)
    return p, vals

def get_max_value(values):
    values=wisps.drop_nan(values)
    if len(values)<1:
        return np.nan
    if np.equal.reduce(values):
        return np.nanmean(values)
    if len(values)>=1:
        kernel= stats.gaussian_kde(distr, bw_method=0.2)
        #kernel = wisps.kernel_density(values)
        height = kernel.pdf(np.linspace(20, 25, 100))
        mode_value = values[np.argmax(height)]
        print (mode_value)
        return float(mode_value)
    
def get_mag_limit(pnt, key, mags):
    #fit for less than 50
    maglt=np.nan
    survey= 'wisps'
  
    #leave 3d hst alone
    if (not pnt.name.lower().startswith('par')): 
        survey='hst3d'
        if (key=='F110'): 
            return maglt
        else:
            maglt=get_max_value(mags)
            print (pnt.imag_exptime, maglt)
            return maglt
            
    if pnt.name.lower().startswith('par'): 
        if (len(mags) < MAG_LIMITS['ncutoff']):
            magpol=MAG_LIMITS[survey][key][0]
            magsctt=MAG_LIMITS[survey][key][1]
            maglt=np.random.normal(magpol(np.log10(pnt.imag_exptime)), magsctt)
            return maglt

        #things aove 50 objects
        if (len(mags) >= MAG_LIMITS['ncutoff']): 
            maglt=get_max_value(mags)
            return maglt

In [ ]:
fig, ax=plt.subplots()
p, m=get_random_pointing('hst3d', 'F140')
plt.hist(m, density=True)
sns.kdeplot(m, ax=ax)
plt.axvline(get_mag_limit(p, 'F140', m), c='k')

In [ ]:
#

In [ ]:
#magpolw=wispsim.MAG_LIMITS[survey][][0]
#magpolh=wispsim.MAG_LIMITS[survey][key][0]

#magsctt=MAG_LIMITS[survey][key][1]

#maglt=np.nanmean(np.random.normal(magpol(np.log10(pnt.exposure_time)), magsctt, 100))

In [ ]:
mag_lts_df['survey']=mag_lts_df['survey'].str.upper()
mag_lts_df=mag_lts_df.rename(columns={"survey": 'SURVEY'})


In [ ]:
import seaborn as sns

In [ ]:
wisps_pnts=[x for x in pnts if x.name.startswith('par')]

In [ ]:
hst3d_pnts=[x for x in pnts if not x.name.startswith('par')]

In [ ]:
pols={'wisps':{}, 'hst3d':{}}

In [ ]:
#wisps.MAG_LIMITS

In [ ]:
pnt_dicts={'wisps':wisps_pnts, 'hst3d':hst3d_pnts}

In [ ]:
import scipy

In [ ]:
def get_random_pointing(survey, key):
    pntsx=[x for x in pnts if x.survey.startswith(survey)]
    p=pntsx[np.random.randint(len(pntsx))]
    vals=wisps.drop_nan(np.vstack(p.mags[key])[:,0])
    print (p)
    return p, vals

def draw_stuff(survey='wisp', key='F140'):
    p, vals=get_random_pointing(survey, key)
    while np.isnan(vals).all():
        p, vals=get_random_pointing(survey, key)
    return p, vals

In [ ]:
fig, ax=plt.subplots(figsize=(12, 4), ncols=3)

p0, vals0=draw_stuff(survey='wisp', key='F110')
vals0=wisps.drop_nan(vals0)
frq0, edges0=np.histogram(vals0, bins=int(np.ptp(vals0)/0.5), normed=True)
grid0=np.linspace(vals0.min(), vals0.max(),1000)
kde0=wisps.kernel_density(vals0).pdf(grid0)
ax[0].plot(grid0, kde0, c='b')
ax[0].bar(edges0[:-1], frq0, width=np.diff(edges0), align="edge", label='0.5 hist', color='#7FDBFF', 
        edgecolor='#7FDBFF', fill=True)
#ax[0].axvline(p0.mag_limits['F110'], c='#2ECC40', linestyle='--', label='kde')
ax[0].axvline(wispsim.get_max_value(vals0))

p1, vals1=draw_stuff(survey='hst3d', key='F140')
vals1=wisps.drop_nan(vals1)
frq1, edges1=np.histogram(vals1, bins=int(np.ptp(vals1)/0.5), normed=True)
grid1=np.linspace(vals1.min(), vals1.max(),1000)
kde1=wisps.kernel_density(vals1).pdf(grid1)
ax[1].plot(grid1, kde1, c='b')
ax[1].bar(edges1[:-1], frq1, width=np.diff(edges1), align="edge", label='0.5 hist', color='#7FDBFF', 
        edgecolor='#7FDBFF', fill=True)
#ax[1].axvline(p1.mag_limits['F140'], c='#2ECC40', linestyle='--', label='kde')
ax[1].axvline(wispsim.get_max_value(vals1))

p2, vals2=draw_stuff(survey='hst3d', key='F160')
vals2=wisps.drop_nan(vals2)
frq2, edges2=np.histogram(vals2, bins=int(np.ptp(vals2)/0.5), normed=True)
kde2=wisps.kernel_density(vals2).pdf(vals2)
ax[2].plot(vals2[np.argsort(vals2)], kde2[np.argsort(vals2)], c='b')
ax[2].bar(edges2[:-1], frq2, width=np.diff(edges2), align="edge", label='0.5 hist', color='#7FDBFF', 
        edgecolor='#7FDBFF', fill=True)
#ax[2].axvline(p2.mag_limits['F160'], c='#2ECC40', linestyle='--', label='kde')
ax[2].axvline(wispsim.get_max_value(vals2))



ax[0].set_xlabel('F110W')
ax[0].set_title(p0.name.upper()+ ' N= {}'.format(len(vals0)))

ax[1].set_xlabel('F140W')
ax[1].set_title(p1.name.upper() + ' N= {}'.format(len(vals1)))

ax[2].set_xlabel('F160W')
ax[2].set_title(p2.name.upper() + ' N= {}'.format(len(vals2)))

plt.savefig(wisps.OUTPUT_FIGURES+'/mag_limit_illustration.pdf')

In [ ]:
#plt.errorbar(volumes[0][0], volumes[0][0], fmt='o', mec='none')